<a href="https://colab.research.google.com/github/dowrave/RoadToImageSeg_GAN/blob/main/220509_Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[텐서플로우 튜토리얼 : 신경 스타일 전이](https://www.tensorflow.org/tutorials/generative/style_transfer?hl=ko)

### 2015년 논문을 따르는 방식
- 이 알고리즘은 이미지 콘텐츠를 특정 스타일로 최적화시키는 방식이다.
- 콘텐츠 이미지 + 스타일 참조 이미지 -> 콘텐츠 유지 & 스타일 참조 이미지의 화풍으로 채색한 느낌의 최적화 기술
- 최근의 CycleGan 등은 모델이 변이된 이미지를 직접 생성하게 만듦 
  - 생성 알고리즘이 스타일 전이 알고리즘보다 1000배 가까이 빠름


In [ ]:
import tensorflow as tf
import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor * 255
  tensor = np.array(tensor, dtype = np.uint8)
  if np.ndim(tensor) > 3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

## 이미지 다운
- 스타일, 컨텐츠 별도

In [ ]:
# 둘 다 1장의 이미지네?
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 
                                       'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('kandinsky5.jpg', 
                                     'https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

## 입력 시각화
- 이미지 불러오는 함수 정의 & 최대 이미지 크기 512개 픽셀

In [ ]:
# img = tf.random.normal([60, 50, 100, 120], dtype = tf.float32)
# # print(img.shape)
# # tf.shape(img[:-1])

In [ ]:
# print(tf.shape(img)[:-1])
# shape = tf.cast(tf.shape(img)[:-1], tf.float32)
# max(shape)

In [ ]:
def load_img(path_to_img):
  max_dim = 512 # 최대 이미지 크기 : 512픽셀

  # 파일 읽기 -> 채널 3개로 디코딩하기(RGB?) -> 데이터 타입 변환하기
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels = 3)
  img = tf.image.convert_image_dtype(img, tf.float32)

 # tf.cast(a, b) : a에 b함수를 가함
  shape = tf.cast(tf.shape(img)[:-1], # tf.shape(img)[:-1] : shape로 갖는 (a,b,c,..z)가 있다면 z 뺀 나머지 값들임
                           tf.float32)
  long_dim = max(shape) # 그 중 최댓값 : 보통은 샘플 수가 제일 많기 때문에 max함수를 지정함
  scale = max_dim / long_dim # 512 / 이미지 가로 세로 중 넓은 거

  new_shape = tf.cast(shape * scale, tf.int32) 

  img = tf.image.resize(img, new_shape) 
  img = img[tf.newaxis, :]
  return img

In [ ]:
# # # 함수 뜯어보기
# max_dim = 512
# img = tf.io.read_file(content_path) # 이 자체로는 사용할 수 없음(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`...)
# img = tf.image.decode_image(img, channels = 3) # decode는 각 픽셀의 RGB값(채널3)을 드러냄
# img = tf.image.convert_image_dtype(img, tf.float32)
# print(img.shape) # (577, 700, 3) : 577, 700은 이미지 해상도?

# shape = tf.cast(tf.shape(img)[:-1], tf.float32) # (577, 700)
# long_dim = max(shape) # 700
# scale = max_dim / long_dim # 이거 왜 나누는거임? : 0.7314

# new_shape = tf.cast(shape * scale , tf.int32)
# new_shape # [422, 512]

# img = tf.image.resize(img, new_shape)
# # print(img.shape)
# img = img[tf.newaxis, :] # 새로운 축을 위치에 따라 추가함. 
# # print(img.shape) # (1, 422, 512, 3)
# print(img)
# tf.squeeze(img, axis = 0)

In [ ]:
# 이미지 출력 함수 정의
def imshow(image, title = None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis = 0) # squeeze : 크기 1인 차원을 제거(tf.newaxis로 추가된 차원을 없앰) / axis = 0 : 0번째 축에 대해서만 squeeze 실행
  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## 콘텐츠, 스타일 표현 정의하기
- CNN의 입력층 쪽은 저차원적 특성, 깊어질수록 고차원적 특성임
- 여기선 사전학습된 이미지 분류 네트워크 VGG19를 사용함
- 중간층은 이미지에서 컨텐츠, 스타일 표현 정의에 필요함
- 스타일 전이 알고리즘은 입력 이미지에 대해 중간층에서 콘텐츠 - 스타일에 해당하는 타깃 표현들을 일치시키려 할 것

In [ ]:
x = tf.keras.applications.vgg19.preprocess_input(content_image * 255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top = True, weights = 'imagenet')
predictions_probabilities = vgg(x)
predictions_probabilities.shape

In [ ]:
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(predictions_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

In [ ]:
vgg = tf.keras.applications.VGG19(include_top = False, weights = 'imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

In [ ]:
content_layers = ['block5_conv2']

style_layers = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1',
                ]
                
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

### 중간 출력으로 스타일, 컨텐츠 표현 정의하기
- 고수준의 이미지 분류 : 네트워크의 이미지 이해가 필수
  - 이는 미가공 이미지를 특성에 대한 복합적인 이해로 변환하는 내부 표현을 만드는 작업이 들어감
  - 또한, 배경잡음 & 기타 잡음에 관계 없이 불변성, 특징 포착 가능
- 따라서 중간층은 복합 특성 추출기의 역할을 수행한다

## 함수형 API로 중간층에 접근하기

In [ ]:
def vgg_layers(layer_names):
  """중간층의 출력값 -> 배열로 반환하는 vgg 모델 만들기"""
  vgg = tf.keras.applications.VGG19(include_top = False, weights = 'imagenet')
  vgg.trainable = False

  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs) # input에 []를 씌운 건 2차원이 필요해서?
  return model

In [ ]:
# 모델 생성
style_extractor = vgg_layers(style_layers) # vgg의 층의 이름을 가진 모델 생성
style_outputs = style_extractor(style_image * 255) # 모델에 input 데이터를 통과시킴

# 각 층의 출력에 대한 통계량
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("크기 : ", output.numpy().shape)
  print("최솟값 : ", output.numpy().min())
  print("최댓값 : ", output.numpy().max())
  print("평균 : ", output.numpy().mean())

## 스타일 계산하기
- 콘텐츠는 중간 층의 feature map의 값들로 표시된다
- 스타일은 각 특성맵의 평균 & 피쳐맵들 사이의 상관관계로 표시되며, <b>그람 행렬</b>은 이를 담고 있음. 자세한 건 해당 튜토리얼 참조
- `tf.linalg.einsum`으로 그람 함수 계산 가능

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc, bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
  return result/(num_locations)

## 스타일, 컨텐츠 추출하기

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg = vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    """input : 0과 1 사이"""
    inputs = inputs * 255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                      outputs[self.num_style_layers:])
    
    style_outputs = [gram_matrix(style_output) for style_output in style_outputs]

    content_dict = {content_name : value for content_name, value in zip(self.content_layers, content_outputs)}

    style_dict = {style_name : value for style_name , value in zip(self.style_layers, style_outputs)}

    return {"content" : content_dict, 'style' : style_dict}
    

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))
print('스타일 : ')
for name, output in sorted(results['style'].items()):
  print(" ", name)
  print("크기 : ", output.numpy().shape)
  print("최솟값 : ", output.numpy().min())
  print("최댓값 : ", output.numpy().max())
  print("평균 : ", output.numpy().mean())
print('-'*40)
print('콘텐츠 : ')
for name, output in sorted(results['content'].items()):
  print(" ", name)
  print("크기 : ", output.numpy().shape)
  print("최솟값 : ", output.numpy().min())
  print("최댓값 : ", output.numpy().max())
  print("평균 : ", output.numpy().mean())

## 경사하강법
- 입력 이미지의 평균제곱오차 계산 후 가중합 구함

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']


In [ ]:
image = tf.Variable(content_image)

In [ ]:
# 픽셀 값 0과 1로 클리핑
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min = 0.0, clip_value_max = 1.0)

In [ ]:
# 옵티마이저 : Adam도 충분히 적합하다
opt = tf.optimizers.Adam(learning_rate = 0.02, beta_1 = 0.99, epsilon = 1e-1)


In [ ]:
# 최적화 진행 : 전체 오차는 컨텐츠와 스타일 오차의 가중 합
style_weight = 1e-2
content_weight = 1e4

In [ ]:
def style_content_loss(outputs):
  style_outputs = outputs['style']
  content_outputs = outputs['content']

  style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name]) ** 2) for name in style_outputs.keys()])
  style_loss *= style_weight / num_style_layers

  content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name]) ** 2) for name in content_outputs.keys()])
  content_loss *= content_weight / num_content_layers

  loss = style_loss + content_loss
  return loss

In [ ]:
# tf.GradientTape로 이미지 업데이트
@tf.function
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

In [ ]:
train_step(image)
train_step(image)
train_step(image)

In [ ]:
# # tensor_to_image 분석

# # print(image.shape) # image : 0 ~ 1 사이의 값을 가진 3차원 텐서
# image = image * 255
# image = np.array(image, dtype = np.uint8)
# # print(image)
# # print('-'*40)
# if np.ndim(image) > 3: 
#   assert image.shape[0] == 1 # 이미지의 차원이 3을 넘으면 0번째 차원의 값은 1일 것이며
#   image = image[0] # 이건 나머지 3개 차원만 보겠다는 말과 동일함
# # image2 = image[tf.newaxis, :]
# # print(image2[0])

# print(image[0]) 
# print(image.shape)
# PIL.Image.fromarray(image) # 배열 -> 이미지로

In [ ]:
import time
start = time.time()
epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end = '')
  display.clear_output(wait = True)
  display.display(tensor_to_image(image))
  print("훈련 스텝 {}".format(step))

end = time.time()
print(end - start)

## 총 변위 손실
- 이 방식의 단점 : 고주파 아티팩트가 다수 존재함
- 이를 위해 규제 항을 추가한다. 변형된 오차 값을 총 변위 손실(Total Variation Loss)이라고 함

In [ ]:
def high_pass_x_y(image):
  x_var = image[:, :, 1:, :] - image[:, :, :-1, :]
  y_var = image[:, :, 1:, :] - image[:, :, :-1, :]

  return x_var, y_var

In [ ]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas : Original")

plt.subplot(2, 2, 2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas : Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2, 2, 3)
imshow(clip_0_1(2*y_deltas + 0.5), "Horizontal Deltas : Styled")

plt.subplot(2, 2, 4)
imshow(clip_0_1(2*x_deltas + 0.5), "Vertical Deltas : Styled")


### 고주파 요소는 경계선 탐지기의 일종이다
- sobel dege detector를 사용하면 유사한 출력을 얻을 수 있음

In [ ]:
plt.figure(figsize=(14, 10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1, 2, 1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel - Edges")
plt.subplot(1, 2, 2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel - Edges")

In [ ]:
# 정규화 오차 : 각 값의 절댓값의 합
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))
  

In [ ]:
total_variation_loss(image).numpy()

In [ ]:
# 정보 : 텐서플로우에 이미 있음
tf.image.total_variation(image).numpy()

## 다시 최적화 하기

In [ ]:
total_variation_weight = 30

In [ ]:
@tf.function
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight * tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

In [ ]:
# 최적화할 변수를 다시 초기화
image = tf.Variable(content_image)

In [ ]:
# 함수 재실행
import time
start = time.time()
epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end = '')
  display.clear_output(wait = True)
  display.display(tensor_to_image(image))
  print("훈련 스텝 {}".format(step))

end = time.time()
print(end - start)

In [ ]:
# 결과 저장
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try : 
  from google.colab import files 
except ImportError:
  pass
else:
  files.download(file_name)